<a href="https://colab.research.google.com/github/pedromonteiro18/HistologyDetectron/blob/main/Histology_Detectron_area_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install detectron2

In [ ]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.


#Problem with latest version, use previous commit
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'


# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 274 kB 5.0 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=f40c0e5ac6b44cbcd0e6b7100d1b344cfac37666bfc4b520086f7cb33cf9d368
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.0 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/d

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Run a pre-trained detectron2 model

We first download an image from the COCO dataset:

In [ ]:
im = cv2.imread("/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/All Segments/s33-BAR06-C.png")
cv2_imshow(im)

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
# cv2_imshow(out.get_image()[:, :, ::-1])

# Train on a custom dataset

Register dataset

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
from detectron2.data.datasets import register_coco_instances
try:
  DatasetCatalog.clear()
  register_coco_instances("dataset_train_1", {}, "/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/Annotations/Modified annotations/BAR05-C.json", "/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/All Segments")
  register_coco_instances("dataset_train_2", {}, "/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/Annotations/Modified annotations/coco-BAR07-BAR08.json", "/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/All Segments")
  register_coco_instances("dataset_train_3", {}, "/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/Annotations/Modified annotations/BAR14-C.json", "/content/drive/MyDrive/RevMed AI/Data/Histology/Segments/All Segments")
  
  register_coco_instances("dataset_val", {}, "/content/drive/My Drive/Colab Notebooks/Histology/Histology COCO Test/Histology Test.json", "/content/drive/My Drive/Colab Notebooks/Histology/Histology COCO Test/Images")

  # register_coco_instances("my_dataset_train2", {}, "/content/drive/MyDrive/CellData/train/EvgTrain-8.json", "/content/drive/MyDrive/CellData/train")
  # register_coco_instances("my_dataset_val2", {}, "/content/drive/MyDrive/CellData/val/EvgVal-9.json", "/content/drive/MyDrive/CellData/val")
except AssertionError:
  print("Already registered")


In [ ]:
trainDict_1 = DatasetCatalog.get("dataset_train_1")
trainDict_2 = DatasetCatalog.get("dataset_train_2")
trainDict_3 = DatasetCatalog.get("dataset_train_3")
testDict = DatasetCatalog.get("dataset_train_1")
# trainDict2 = DatasetCatalog.get("my_dataset_train2")
# testDict2 = DatasetCatalog.get("my_dataset_val2")

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [ ]:
# dataset_dicts = trainDict_1
# histology_metadata = MetadataCatalog.get("dataset_train_1")
# print(dataset_dicts)
# for d in random.sample(dataset_dicts, 1):
#     print("d: ", d)
#     print(d["file_name"])
#     img = cv2.imread(d["file_name"])

#     # ---resize--- or colab will crash when trying to view the full image
#     img = cv2.resize(img, (0,0), fx=0.6, fy=0.6) 

#     visualizer = Visualizer(img[:, :, ::-1], metadata=histology_metadata, scale=0.5)
#     out = visualizer.draw_dataset_dict(d)

    # Uncomment below to view an example image
    # cv2_imshow(out.get_image()[:, :, ::-1])

## Train!

Now, let's fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the balloon dataset. It takes ~6 minutes to train 300 iterations on Colab's K80 GPU, or ~2 minutes on a P100 GPU.


In [ ]:
from detectron2.engine import DefaultTrainer

CUDA_LAUNCH_BLOCKING=1 # For debugging purposes
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dataset_train_1","dataset_train_2","dataset_train_3")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2


cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/Colab Notebooks/Histology/model_final.pth")  # path to the model we just trained

cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.BASE_LR = 0.025  # learning rate
cfg.SOLVER.MAX_ITER = 1  # number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 6   # faster training with a smaller batch size, (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8  # number of classes (number of label types)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Inference & evaluation using the trained model
Run inference with the trained model on the dataset. Create a predictor using the model we just trained:



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/Colab Notebooks/Histology/model_final.pth")  # path to the model we just trained

#set the confidence level threshold for inference 
# conf_level = 0.7

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = conf_level  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [ ]:
from detectron2.utils.visualizer import ColorMode
from detectron2.structures.masks import polygon_area
# !pip install tensorflow
import tensorflow as tf
import os

# dataset_dicts = get_balloon_dicts("balloon/val")
dataset_dicts = testDict
for d in random.sample(dataset_dicts, 1):    
    print('Image: ', os.path.basename(d["file_name"]))

    im = cv2.imread(d["file_name"])
    im = cv2.resize(img, (0,0), fx=0.6, fy=0.6) 

    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print('Classes available: ', histology_metadata.thing_classes)
    confident_detections = outputs["instances"][outputs["instances"].scores > conf_level]
    print('Number of instances: ', len(confident_detections))
    # print(int(confident_detections.pred_classes))
    # print('Classes of predicted instances: ', list(map(lambda x: histology_metadata.thing_classes[int(x)], confident_detections.pred_classes)))
    print("Number of confidence detections: ", len(confident_detections))
    for ins in range(len(confident_detections)):
      mask = confident_detections[ins].pred_masks
      with tf.compat.v1.Session() as sess:
        sumT = tf.math.count_nonzero(mask.cpu())
        truePixels = sess.run(sumT)
      # print(histology_metadata.thing_classes[confident_detections[ins].pred_classes] +' : ', 100*truePixels/(mask.size()[1]*mask.size()[2]),'%')

    v = Visualizer(im[:, :, ::-1],
                   metadata=histology_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

We can also evaluate its performance using AP metric implemented in COCO API.

In [ ]:
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("balloon_val", ("bbox", "segm"), False, output_dir="./output/")
# val_loader = build_detection_test_loader(cfg, "balloon_val")
# print(inference_on_dataset(trainer.model, val_loader, evaluator))
# # another equivalent way to evaluate the model is to use `trainer.test`

# Other types of builtin models

In [ ]:
# # Inference with a panoptic segmentation model
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
# predictor = DefaultPredictor(cfg)
# panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
# v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
# out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
# cv2_imshow(out.get_image()[:, :, ::-1])